# Probability & Bayesian Networks

In [1]:
from probability import *
import itertools

# Set shortened constants for True/False
T, F = True, False

## Part 1 - Probability Distribution - Basics
A) You have an unbiased six-sided dice a . The die is rolled twice to generate the outcomes X1 and X2. Using the code made available from the AIMA data repo, calculate the probability of generating SnakeEyes (1,1 - each 1 is rolled in succession rather than two dice together) and print out the probability:<br>
**Expected output:** “Probability of Snake Eyes is **X**” where X is the probability

In [2]:
# Assign each side of a dice their probability
p = ProbDist(freqs={'1': 1, '2': 1, '3': 1, '4': 1, '5': 1, '6': 1})

# Create joint probability distribution object with two die
variables = ['X1', 'X2']
j = JointProbDist(variables)

# For two rolls of a die, get all possible combinations
dice_sides = [1, 2, 3, 4, 5, 6]
two_dice_rolls = [p for p in itertools.product(dice_sides, repeat=2)]

# For each combination calculate the probability dependent on the
# probability of each side of the dice from p, assign that combo and
# probability to joint probability object
for combo in two_dice_rolls:
    j[combo[0], combo[1]] = p[str(combo[0])] * p[str(combo[1])]

# Output the probability of rolling two ones
print("Probability of Snake Eyes is: {}".format(j[1, 1]))

Probability of Snake Eyes is: 0.027777777777777776


## Part 2 - Constructing a Bayesian Network
Construct a Bayes net using the BayesNet class for the following scenario:

You have a daily commute to work, a number of considerations that can affect your commute. You also have a temperamental boss, if your late he/she will typically berate you over the phone which will leave you feeling dejected for the day. Sometimes you use the Motorway to make up time and avoid being late.

**Variables:** Traffic (T), Rain (R), Motorway (M), Late (L), BossCalls (B)<br>
**Network Topology:**
- Sometimes you decide to take the Motorway
- Rain can result in you being late
- Traffic can result in you being late
- Being late can cause your boss to call

A) Draw the Bayesian Network

![Late Bayes Network](https://i.ibb.co/Bjtsq2P/Blank-Diagram-1.jpg)

B) Using the BayesNode code from the AIMA repository create a Bayesian Network (BN) based on this scenario. 

In [3]:
# Construct the Bayes Network, parent nodes first to be defined, then all
# subsequent children
workCommute = BayesNet([
    ('Rain', '', 0.41),
    ('Traffic', '', 0.15),
    ('MotorWay', '', 0.01),
    ('Late', 'Rain Traffic MotorWay',
     {(T, T, T): 0.8,
      (T, T, F): 0.98,
      (T, F, T): 0.2,
      (T, F, F): 0.3,
      (F, T, T): 0.25,
      (F, T, F): 0.24,
      (F, F, T): 0.001,
      (F, F, F): 0.05}),
    ('BossCalls', 'Late', {T: 0.8, F: 0.1})])

C) Write a query to output the CPT for the “Late” node.

In [4]:
def print_table(table):
    """
    Print a formatted table.

    :param table: (list) Nested list, each list represents a row in the table.
    """
    longest_cols = [
        (max([len(str(row[i])) for row in table]) + 3)
        for i in range(len(table[0]))
    ]
    row_format = "".join(
        ["{:>" + str(longest_col) + "}" for longest_col in longest_cols])
    counter = 0
    for row in table:
        if counter == 0:
            dash = '-' * 48
            print(dash)
            print(row_format.format(*row))
            print(dash)
        else:
            print(row_format.format(*row))
        counter += 1
        
# Calculate CPT for being late
late_cpt = workCommute.variable_node('Late').cpt

# Get headings and rows for CPT, output using print_table()
headings = ['Rain', 'Traffic', 'Motorway', 'Probability Late']
table = list()
table.append(headings)
# For each combination in the CPT, extract tuple values and assign
# probability, append to table list for output to table
for k, v in late_cpt.items():
    row = [str(k[0]), str(k[1]), str(k[2]), v]
    table.append(row)
# Output table
print_table(table)

------------------------------------------------
    Rain   Traffic   Motorway   Probability Late
------------------------------------------------
    True      True       True                0.8
    True      True      False               0.98
    True     False       True                0.2
    True     False      False                0.3
   False      True       True               0.25
   False      True      False               0.24
   False     False       True              0.001
   False     False      False               0.05


D) Using the BN from Qii, write the python query to answer the following queries:
- You took the Motorway
- The boss does not call given that you are late
- You are late when its raining & there is traffic as you took the Motorway

In [5]:
# Calculate probability that the MotorWay was taken by invoking True
# probability of variable node 'MotorWay'
print("\nProbability you took the motorway: {}".format(
    workCommute.variable_node('MotorWay').p(T, 'MotorWay')))

# Calculate probability the boss does not call if you are late by False
# probability of variable node 'BossCalls' given that 'Late' is True
print("Probability the boss does not call given that you are late: "
      "{}".format(workCommute.variable_node('BossCalls').p(F, {'Late': T})))

# Calculate probability you are late then its raining, there is traffic,
# and you took the motorway by invoking the variable node 'Late' given
# that Rain/Traffic/MotorWay all resolve to True
print("Probability you are late when it's raining & there is traffic as "
      "you took the motorway: {}".format(
        workCommute.variable_node('Late').p(T, {'Rain': T,
                                                'Traffic': T,
                                                'MotorWay': T})))


Probability you took the motorway: 0.01
Probability the boss does not call given that you are late: 0.19999999999999996
Probability you are late when it's raining & there is traffic as you took the motorway: 0.8


## Part 3 - Exact Inference in Bayesian Network
A) Implement the following queries:<br>
- It is raining when the Boss calls **X**% of the time<br>
- It is not raining when the Boss calls **X**% of the time<br>

In [6]:
# Calculate the probability it is raining when the boss calls
ans_dist_a = enumeration_ask('Rain', {'BossCalls': T}, workCommute)
print("It is raining when the bass calls {:.3f}% of the time".format(
        ans_dist_a[T] * 100))
print("It is raining when the bass calls {:.3f}% of the time".format(
        ans_dist_a[F] * 100))

It is raining when the bass calls 63.101% of the time
It is raining when the bass calls 36.899% of the time


- There is traffic when the Boss calls around **X**% of the time.
- There is no traffic when the Boss calls **X**% of the time

In [7]:
# Calculate the probability of traffic when the boss calls
ans_dist_b = enumeration_ask('Traffic', {'BossCalls': T}, workCommute)
print("\nThere is traffic when the Boss calls {:.3f}% of the time".format(
        ans_dist_b[T] * 100))
print("There is traffic when the Boss calls {:.3f}% of the time".format(
        ans_dist_b[F] * 100))


There is traffic when the Boss calls 29.108% of the time
There is traffic when the Boss calls 70.892% of the time


- I am using the Motorway when the Boss calls around **X**% of the time
- I am not using the Motorway when the Boss calls around **X**% of the time

In [8]:
# Calculate the probability of using the MotorWay when the boss calls
ans_dist_c = enumeration_ask('MotorWay', {'BossCalls': T}, workCommute)
print("\nI am using the Motorway when the Boss calls around {:.3f}% of "
      "the time".format(ans_dist_c[T] * 100))
print("I am not using the Motorway when the Boss calls around {:.3f}% of "
      "the time".format(ans_dist_c[F] * 100))


I am using the Motorway when the Boss calls around 0.805% of the time
I am not using the Motorway when the Boss calls around 99.195% of the time


- The Boss calls when it is raining and there is Traffic around **X**% of the time
- The Boss does not call when it is raining and there is Traffic around **X**% of the time

In [9]:
# Calculate the probability it is raining and there is traffic when the boss calls
ans_dist_d = enumeration_ask('BossCalls', {'Rain': T, 'Traffic': T}, workCommute) 
print("\nThe Boss calls when it is raining and there is Traffic around "
      "{:.3f}% of the time".format(ans_dist_d[T] * 100))
print("The Boss does not calls when it is raining and there is Traffic "
      "around {:.3f}% of the time".format(ans_dist_d[F] * 100))


The Boss calls when it is raining and there is Traffic around 78.474% of the time
The Boss does not calls when it is raining and there is Traffic around 21.526% of the time


### Part 2
Q) Explain how inference by enumeration works? Particularly in relation to your answer for the prior question.

A) Inference by enumeration works by summing out the variables from the joint probability distribution without actually constructing its explicit representation. 

The process involves stating all marginal probabilities needed, determining all the atomic probabilities needed, calculating and combinign them.  General inference queries will have the following attributes:

Observed evidence variables: <br><br>$$E=e1, e1, e2$$<br> 
Query variables, or variables we wish to know the probability of: <br><br>$$Q.$$<br>
Hidden variables, variables that are along for the ride but that we don't care about: <br><br>$$H=h1,h2....hn$$<br>
So the general query structure would be: <br><br>$$P(Q|e1,e2,...,en)$$<br>

Given the general query structure, the simple query in the late for work network 'The Boss calls when it is raining and there is Traffic around **X** % of the time' can be reperesented as: <br><br>$$P(Boss Calls | raining = True, traffic = true)$$<br>

This general query has the following attributes:
- Query: *Boss calls = True*
- Observed evidence variables: *Raining = True, Traffic = True*
- Hidden variables:
    - If the *MotorWay* was taken
    - If we were *late*

Taking what is known results in the following equation:
$$P(B\ |\ r,\ t)$$<br>

Which can be further expanded to give:
$$=\frac{P(r\ |\ B)P(t\ |\ B)P(B)}{P(r)P(t)}$$ <br>

Since we know $\alpha\ = P(r)P(t)$ the equation can be written as:
$$=\alpha\ P(r\ |\ B)P(t\ |\ B)P(B)$$ <br>

And because we can assume B is 1 because B is True, the calculations of determining B are irreleveant the equation can be further shortened to:
$$=\alpha\ P(r)P(t)P(B)$$
$$=\alpha\ P(B, r, t)$$ <br>

The query can be answered using a Bayesian network by computing sums of products of conditional probabilities from the network.  It is possible at this point to sum in the unseen events 'Late (l)' and 'MotorWay (m)':
$$=\alpha\ \Sigma{m}\ \Sigma{l}\ P(B, r, t, m, l)$$ <br>

Following the semantics of Bayesian networks and CPT entries the the following expression emerges (we are only interested in *BossCalls=True*):
$$=\alpha\ \Sigma{m}\ \Sigma{l}\ P(B)P(m)P(l\ | r,t,m)P(B\ | l)$$ <br>

As *BossCalls=True* is our constant it can be moved outside the summations over m and l, and m can be moved outside the summation over l giving:
$$=\alpha\ P(B) \Sigma{m}\ P(m) \Sigma{l}\ P(l\ | r,t,m)P(B\ | l)$$ <br>

This excpression is evaluated by looping through the variables in order, multiplying CPT entries as it goes. For each summation, it is necessary to loop over the variable's possible values.  This is the function of methods *enumerate_ask()* and *enumerate_all()* in probability.py in the AIMA repo.

Using the numbers from the provided CPTs, the equation yields (after normalisation with $\alpha$) a split of 0.99195/0.00805 in favour of the boss calling when it is raining and there is traffic.
